### PROD MODELS
distil-whisper-large-v3-en, gemma2-9b-it, llama-3.3-70b-versatile, llama-3.1-8b-instant, llama-guard-3-8b, llama3-70b-8192, llama3-8b-8192
mixtral-8x7b-32768, whisper-large-v3,whisper-large-v3-turbo
### PREVIEW MODELS
deepseek-r1-distill-llama-70b, llama-3.3-70b-specdec, llama-3.2-1b-preview, llama-3.2-3b-preview, llama-3.2-11b-vision-preview, llama-3.2-90b-vision-preview

In [32]:

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")



In [33]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

# llm.invoke("Hello, how are you?")

In [41]:
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)
    

|The| color| of| the| sky| can| appear| different| depending| on| the| time| of| day|,| atmospheric| conditions|,| and| location|.| Here| are| some| common| answers|:

|1|.| **|Blue|**:| During| the| daytime|,| when| the| sun| is| overhead|,| the| sky| typically| appears| a| shade| of| blue| to| our| eyes|.| This| is| because| the| Earth|'s| atmosphere| sc|atters| sunlight| in| all| directions|,| and| shorter| (|blue|)| wavelengths| are| scattered| more| than| longer| (|red|)| wavelengths|,| giving| the| sky| its| blue| color|.
|2|.| **|Golden|/H|oney|**:| During| sunrise| and| sunset|,| the| sky| can| take| on| hues| of| gold|,| orange|,| pink|,| and| red|.| This| is| due| to| the| scattering| of| light| by| atmospheric| particles|,| which| favors| longer| wavelengths| like| red| and| orange|.
|3|.| **|Gray|**:| On| over|cast| days|,| the| sky| can| appear| a| dull| gray| or| white|,| as| the| sunlight| is| scattered| in| all| directions| by| the| cloud| cover|.
|4|.| **|Black|**:| At

In [43]:
chunks[4]

AIMessageChunk(content=' the', additional_kwargs={}, response_metadata={}, id='run-c554bf24-9d9c-457b-bf64-11eab5601110')

In [44]:
chunks = []
async for chunk in model.astream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

|What| a| great| question|!

|The| answer| is| not| a| simple| one|,| as| it| depends| on| the| time| of| day|,| atmospheric| conditions|,| and| the| observer|'s| location|.| Here| are| some| different| answers|:

|1|.| **|During| the| day|**:| When| the| sun| is| overhead|,| the| sky| appears| blue| to| our| eyes| because| of| a| phenomenon| called| Ray|leigh| scattering|.| This| is| when| shorter| (|blue|)| wavelengths| of| light| are| scattered| more| than| longer| (|red|)| wavelengths| by| the| tiny| molecules| of| gases| in| the| atmosphere|,| such| as| nitrogen| and| oxygen|.| This| scattering| effect| gives| the| sky| its| blue| color|.
|2|.| **|At| sunrise| and| sunset|**:| During| these| times|,| the| sun| is| lower| in| the| sky|,| and| the| light| it| emits| has| to| travel| longer| distances| through| the| atmosphere|.| As| a| result|,| much| of| the| shorter| wavelength| blue| light| is| scattered| away|,| leaving| mainly| longer| wavelength| red| and| orange| light| to| r

In [45]:
chunks[1]

AIMessageChunk(content='What', additional_kwargs={}, response_metadata={}, id='run-5fff3a6d-ea9f-4ef8-ae4b-2995310e5186')

In [46]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4]

AIMessageChunk(content='What a great question', additional_kwargs={}, response_metadata={}, id='run-5fff3a6d-ea9f-4ef8-ae4b-2995310e5186')

# Chain using Langchain expression language

In [47]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

model = ChatGroq(model="llama3-8b-8192")

parser = StrOutputParser()

chain = prompt | model | parser

async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

|Why| did| the| par|rot| go| to| the| doctor|?

|Because| it| had| a| f|owl| cough|!||

# Working with Input Streams

In [48]:
from langchain_core.output_parsers import JsonOutputParser

chain = (
    model | JsonOutputParser()
)  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models
async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 652}]}
{'countries': [{'name': 'France', 'population': 652735}]}
{'countries': [{'name': 'France', 'population': 65273539}]}
{'countries': [{'name': 'France', 'population': 65273539}, {}]}
{'countries': [{'name': 'France', 'population': 65273539}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 65273539}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 65273539}, {'name': 'Spain', 'population': 467}]}
{'countries': [{'name': 'France', 'population': 65273539}, {'name': 'Spain', 'population': 467547}]}
{'countries': [{'name': 'France', 'population': 65273539}, {'name': 'Spain', 'population': 46754713}]}
{'countries': [{'name': 'France', 'population': 65273539}, {'name': 'Spain', 'population': 46754713}, {}]}
{'countries': [{'name': 'France', 'population': 65273539}, {'name': 'Spain', 'population': 467

In [49]:
from langchain_core.output_parsers import (
    JsonOutputParser,
)


# A function that operates on finalized inputs
# rather than on an input_stream
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = model | JsonOutputParser() |  _extract_country_names

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, end="|", flush=True)

['France', 'Spain', 'Japan']|

In [50]:
from langchain_core.output_parsers import JsonOutputParser


async def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    async for input in input_stream:
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)


chain = model | JsonOutputParser() | _extract_country_names_streaming

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(text, end="|", flush=True)

France|Spain|Japan|

In [51]:
#!pip install faiss-cpu
#! pip install langchain_community
#! pip install langchain_openai
#! pip install langchain_huggingface
#! pip install -U sentence-transformers
#! pip install faiss-cpu
# ! pip install langchain_openai


In [52]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings


In [53]:

from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")



In [54]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding=embeddings_model,
)
retriever = vectorstore.as_retriever()

# chunks = [chunk for chunk in retriever.stream("where did harrison work?")]
# chunks

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)
for chunk in retrieval_chain.stream(
    "Where did harrison work? " "Write 3 made up sentences about this place."
):
    print(chunk, end="|", flush=True)

|According| to| the| context|,| Harrison| worked| at| Kens|ho|.

|Here| are| three| made|-up| sentences| about| Kens|ho|:

|K|ens|ho| is| a| trendy| startup| located| in| the| heart| of| Silicon| Valley|,| specializing| in| AI|-powered| analytics|.| The| office| space| features| a| rooftop| garden| where| employees| can| relax| and| enjoy| the| views| of| the| surrounding| city|scape|.| Kens|ho|'s| state|-of|-the|-art| facilities| are| equipped| with| the| latest| technology|,| making| it| an| ideal| workplace| for| innov|ators| and| thinkers| like| Harrison|.||

In [55]:
events = []
async for event in model.astream_events("hello", version="v2"):
    events.append(event)

In [57]:
len(events)

29

In [58]:
events[-1]

{'event': 'on_chat_model_end',
 'data': {'output': AIMessageChunk(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'finish_reason': 'stop'}, id='run-eccd533c-8769-4119-9fa0-a69b88676bbd', usage_metadata={'input_tokens': 11, 'output_tokens': 26, 'total_tokens': 37})},
 'run_id': 'eccd533c-8769-4119-9fa0-a69b88676bbd',
 'name': 'ChatGroq',
 'tags': [],
 'metadata': {'ls_provider': 'groq',
  'ls_model_name': 'llama3-8b-8192',
  'ls_model_type': 'chat',
  'ls_temperature': 0.7},
 'parent_ids': []}

In [59]:
chain = (
    model | JsonOutputParser()
)  # Due to a bug in older versions of Langchain, JsonOutputParser did not stream results from some models

events = [
    event
    async for event in chain.astream_events(
        "output a list of the countries france, spain and japan and their populations in JSON format. "
        'Use a dict with an outer key of "countries" which contains a list of countries. '
        "Each country should have the key `name` and `population`",
        version="v2",
    )
]

In [62]:
len(events)

154

In [63]:
num_events = 0

async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
    version="v2",
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(f"Parser chunk: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 30:
        # Truncate the output
        print("...")
        break

Chat model chunk: ''
Chat model chunk: 'Here'
Chat model chunk: ' is'
Chat model chunk: ' the'
Chat model chunk: ' JSON'
Chat model chunk: ' output'
Chat model chunk: ':\n'
Chat model chunk: '``'
Chat model chunk: '`\n'
Chat model chunk: '{\n'
Parser chunk: {}
Chat model chunk: ' '
Chat model chunk: ' "'
Chat model chunk: 'countries'
Chat model chunk: '":'
Chat model chunk: ' [\n'
Parser chunk: {'countries': []}
Chat model chunk: '   '
Chat model chunk: ' {\n'
Parser chunk: {'countries': [{}]}
Chat model chunk: '     '
Chat model chunk: ' "'
Chat model chunk: 'name'
Chat model chunk: '":'
Chat model chunk: ' "'
...


In [64]:
chain = model.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
    version="v2",
    include_names=["my_parser"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_parser_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'my_parser', 'tags': ['seq:step:2'], 'run_id': '230389e0-9d88-40ab-bee1-6070052eab6e', 'metadata': {}, 'parent_ids': ['9df8052b-7bdc-406c-9245-9f23afee4f97']}
{'event': 'on_parser_stream', 'run_id': '230389e0-9d88-40ab-bee1-6070052eab6e', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {}}, 'parent_ids': ['9df8052b-7bdc-406c-9245-9f23afee4f97']}
{'event': 'on_parser_stream', 'run_id': '230389e0-9d88-40ab-bee1-6070052eab6e', 'name': 'my_parser', 'tags': ['seq:step:2'], 'metadata': {}, 'data': {'chunk': {'countries': []}}, 'parent_ids': ['9df8052b-7bdc-406c-9245-9f23afee4f97']}
{'event': 'on_parser_stream', 'run_id': '230389e0-9d88-40ab-bee1-6070052eab6e', 'name': 'my_parse

In [65]:
chain = model.with_config({"run_name": "model"}) | JsonOutputParser().with_config(
    {"run_name": "my_parser"}
)

max_events = 0
async for event in chain.astream_events(
    'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`',
    version="v2",
    include_types=["chat_model"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_chat_model_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'model', 'tags': ['seq:step:1'], 'run_id': '1e3a4d13-6c4a-495e-bc70-96c03547362d', 'metadata': {'ls_provider': 'groq', 'ls_model_name': 'llama3-8b-8192', 'ls_model_type': 'chat', 'ls_temperature': 0.7}, 'parent_ids': ['faa9e968-f78c-44e6-8001-7eec291e0255']}
{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-1e3a4d13-6c4a-495e-bc70-96c03547362d')}, 'run_id': '1e3a4d13-6c4a-495e-bc70-96c03547362d', 'name': 'model', 'tags': ['seq:step:1'], 'metadata': {'ls_provider': 'groq', 'ls_model_name': 'llama3-8b-8192', 'ls_model_type': 'chat', 'ls_temperature': 0.7}, 'parent_ids': ['faa9e968-f78c-44e6-8001-7eec291e0255']}
{'event':

In [66]:
chain = (model | JsonOutputParser()).with_config({"tags": ["my_chain"]})

max_events = 0
async for event in chain.astream_events(
    'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`',
    version="v2",
    include_tags=["my_chain"],
):
    print(event)
    max_events += 1
    if max_events > 10:
        # Truncate output
        print("...")
        break

{'event': 'on_chain_start', 'data': {'input': 'output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`'}, 'name': 'RunnableSequence', 'tags': ['my_chain'], 'run_id': '129ee5e1-dfb8-4c6b-a46d-cb782ccb2037', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='output a list of the countries france, spain and japan and their populations in JSON format. Use a dict with an outer key of "countries" which contains a list of countries. Each country should have the key `name` and `population`', additional_kwargs={}, response_metadata={})]]}}, 'name': 'ChatGroq', 'tags': ['seq:step:1', 'my_chain'], 'run_id': '3d6b80fb-53ba-47ab-bf91-1d198850ee18', 'metadata': {'ls_provider': 'groq', 'ls_model_name': 'llama3-8b-8192', 'ls_model_type': 'chat', 'ls_temperature

In [67]:
# Function that does not support streaming.
# It operates on the finalizes inputs rather than
# operating on the input stream.
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names


chain = (
    model | JsonOutputParser() | _extract_country_names
)  # This parser only works with OpenAI right now

In [68]:
async for chunk in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(chunk, flush=True)

['France', 'Spain', 'Japan']


In [69]:
num_events = 0

async for event in chain.astream_events(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
    version="v2",
):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(
            f"Chat model chunk: {repr(event['data']['chunk'].content)}",
            flush=True,
        )
    if kind == "on_parser_stream":
        print(f"Parser chunk: {event['data']['chunk']}", flush=True)
    num_events += 1
    if num_events > 30:
        # Truncate the output
        print("...")
        break

Chat model chunk: ''
Chat model chunk: 'Here'
Chat model chunk: ' is'
Chat model chunk: ' the'
Chat model chunk: ' JSON'
Chat model chunk: ' output'
Chat model chunk: ' you'
Chat model chunk: ' requested'
Chat model chunk: ':\n'
Chat model chunk: '``'
Chat model chunk: '`\n'
Chat model chunk: '{\n'
Parser chunk: {}
Chat model chunk: ' '
Chat model chunk: ' "'
Chat model chunk: 'countries'
Chat model chunk: '":'
Chat model chunk: ' [\n'
Parser chunk: {'countries': []}
Chat model chunk: '   '
Chat model chunk: ' {\n'
Parser chunk: {'countries': [{}]}
Chat model chunk: '     '
Chat model chunk: ' "'
Chat model chunk: 'name'
Chat model chunk: '":'
Chat model chunk: ' "'
...


In [70]:
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import tool


def reverse_word(word: str):
    return word[::-1]


reverse_word = RunnableLambda(reverse_word)


@tool
def bad_tool(word: str):
    """Custom tool that doesn't propagate callbacks."""
    return reverse_word.invoke(word)


async for event in bad_tool.astream_events("hello", version="v2"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'bad_tool', 'tags': [], 'run_id': '83c3ff86-327a-4e6c-adb9-86415db44b77', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': 'f58aaf10-507f-4d9f-bf3a-69efdb1f4825', 'metadata': {}, 'parent_ids': ['83c3ff86-327a-4e6c-adb9-86415db44b77']}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': 'f58aaf10-507f-4d9f-bf3a-69efdb1f4825', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['83c3ff86-327a-4e6c-adb9-86415db44b77']}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': '83c3ff86-327a-4e6c-adb9-86415db44b77', 'name': 'bad_tool', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [71]:
@tool
def correct_tool(word: str, callbacks):
    """A tool that correctly propagates callbacks."""
    return reverse_word.invoke(word, {"callbacks": callbacks})


async for event in correct_tool.astream_events("hello", version="v2"):
    print(event)

{'event': 'on_tool_start', 'data': {'input': 'hello'}, 'name': 'correct_tool', 'tags': [], 'run_id': 'a61cce1f-c0f6-4d90-a438-121b7b7ae765', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': 'hello'}, 'name': 'reverse_word', 'tags': [], 'run_id': 'c926aa0a-85b9-4da2-874c-bc32a83dbd18', 'metadata': {}, 'parent_ids': ['a61cce1f-c0f6-4d90-a438-121b7b7ae765']}
{'event': 'on_chain_end', 'data': {'output': 'olleh', 'input': 'hello'}, 'run_id': 'c926aa0a-85b9-4da2-874c-bc32a83dbd18', 'name': 'reverse_word', 'tags': [], 'metadata': {}, 'parent_ids': ['a61cce1f-c0f6-4d90-a438-121b7b7ae765']}
{'event': 'on_tool_end', 'data': {'output': 'olleh'}, 'run_id': 'a61cce1f-c0f6-4d90-a438-121b7b7ae765', 'name': 'correct_tool', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [72]:
from langchain_core.runnables import RunnableLambda


async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2


reverse_and_double = RunnableLambda(reverse_and_double)

await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': '78621d92-0b6d-4815-9c1a-f3065d532666', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': '78621d92-0b6d-4815-9c1a-f3065d532666', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'data': {'chunk': '43214321'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': '78621d92-0b6d-4815-9c1a-f3065d532666', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'parent_ids': []}


In [73]:
from langchain_core.runnables import chain


@chain
async def reverse_and_double(word: str):
    return await reverse_word.ainvoke(word) * 2


await reverse_and_double.ainvoke("1234")

async for event in reverse_and_double.astream_events("1234", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': '1234'}, 'name': 'reverse_and_double', 'tags': [], 'run_id': '3ebbf4f1-8fe7-4fac-a03c-6a44abac63de', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': '3ebbf4f1-8fe7-4fac-a03c-6a44abac63de', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'data': {'chunk': '43214321'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': '43214321'}, 'run_id': '3ebbf4f1-8fe7-4fac-a03c-6a44abac63de', 'name': 'reverse_and_double', 'tags': [], 'metadata': {}, 'parent_ids': []}
